# RNA secondary structure

## Introduction

The code you are going to write using the template/guide provided is using dynamic programming to find the secondary structure of RNA. Before attacking the code exercise you must: 
* read the background material (Algorithm Design by Kleinberg & Tardos p.272-278)
* carry-out the formulate step
* apply the obtained recursive formula to a simple example

This is essential since one needs to fully grasp the mathematical problem before coding.  

Once this is done, you are ready to start. When you start any journey, you should have your destination in mind, the same with coding, you should know what you want the code to do, in particular which information to give the code and the result. 
For the specific case, one wants to give a sequence of RNA bases and be returned how they pair (see Kleinberg & Tardos).

If the journey is long, you also plan some stops along the way. The same for a code project of a certain complexity, one betters break it into logical/functional units.
For the specific case, from your experience of dynamic programming you know that there is a tabulate part and a look-up part. 

After implementing each bit of code, remember to test that bit of code does exactly what you expected.

We start from some essential on strings that we need since the base sequence in input is a string.

## Some essentials on strings and an important function

The RNA bases are indicated with characters: A, U, G, C and the RNA primary structure is given as a string of these characters.  To be able to implement the rules to form base pairs (allowable pairs are AU or UA or CG or GC) we need to introduce python strings (here a suggested tutorial on strings: https://www.w3schools.com/python/python_strings.asp). Strings in python are surrounded by either single quotation marks, or double quotation marks.
Here, for example, we assign to the variable ``rna`` a string:

In [2]:
rna = 'UUCGAUAACGC'

For practical use in this project, strings are like lists of characters. So, rna[2] for example is the third (since as always in python, numbering starts from 0) character, or base,  of ``rna``, that is ``C``, which you can verify by executing:

In [3]:
rna[2]

'C'

We can also concatenate strings, using +. Here below, the third base (C) is contacatened with the tenth base of the rna (G). When executing the code below one should obtain the string ``'CG'``. You will need concatenation later on. So feel free to play around with it if needed.

In [4]:
rna[2]+rna[9]

'CG'

We can also use logic on strings. Later, we would need to determine whether a pair is allowable, that is if it is either AU or UA or CG or GC.

In [5]:
rna[2]+rna[9] == 'CG'

True

In [6]:
rna[2]+rna[8] == 'AU'

False

We will also need to determine how long is a string. This is given by the internal python function ``len``. For example for ``rna`` executing the command below gives 11, the length of ``rna``

In [7]:
len(rna)

11

We will now use what we learned on string into the function that is at the core of the RNA base pairing. As part of the recurrence:
$$\max(1 + OPT(i, t − 1) + OPT(t + 1, j − 1))$$
where the max is taken over $t$ such that the base $b_t$ and base $b_j$ are an allowable base pair. 

The function ``max_over_t`` takes as input:
* the indexes ``i`` and ``j`` that identify the piece of RNA base sequence
* ``M`` the array that contains the optimal value function 
* ``seq`` the string with the RNA base sequence
and returns 
* ``tmax`` the $t$ that maximises  $(1 + OPT(i, t − 1) + OPT(t + 1, j − 1))$
* ``mmax`` the corresponding value of $(1 + OPT(i, t − 1) + OPT(t + 1, j − 1))$

The blueprint for the function is:

``max_over_t(i,j,M,seq)
    initialise mmax and tmax (already done below)
    for t = i, ..., j-4 
        Set pair equal to the t and j bases pair
        if pair is any of 'AU','UA','CG','GC'
            Set m equal to 1 + M[i, t − 1] + M[t + 1, j − 1])
            if m > mmax then
                set mmax equal to m
                set tmax equal to t
     return mmax and tmax``

To set the ``pair`` variable and verify if is a given string remember what we have just seen for string concatanation and logic.  

In [8]:
def successfulPairing(seq, i, j):
    if (seq[i] == 'A' and seq[j] == 'U') or (seq[i] == 'U' and seq[j] == 'A') or (seq[i] == 'G' and seq[j] == 'C') or (seq[i] == 'C' and seq[j] == 'G'):
        return True
    else:
        return False

In [9]:
def getPairs(seq, i, j):
    pairs = []
    for t in range(i, j-4):
        if successfulPairing(seq, t, j):
            pairs.append(t)
    return pairs

In [10]:
import numpy as np
def max_over_t(i,j,M,seq):
    mmax = 0
    tmax = -1 

    pairs = getPairs(seq, i, j)
    for t in pairs:
        m = 1 + M[i, max(0,t-1)] + M[t+1, j-1]
        if m > mmax:
            mmax = m
            tmax = t
    
    return tmax,mmax

## Tabulate... 

We start by tabulating the optimal value function for each stage and state. This is done by the function ``Tabulate_RNA`` that takes as input ``seq`` the RNA sequence and 

We refer to the code blueprint in the Kleinberg and Tardos (p.277)

  ```
    Tabulate_RNA(seq)
    Initialize OPT (i, j) = 0 whenever i ≥ j − 4
    For k = 5, 6, . . . , n − 1
        For i = 1, 2, . . . n − k
            Set j = i + k
            Compute OPT (i, j) using the recurrence in (6.13)
        Endfor
    Endfor
    Return OPT (1, n)
  ```
    
where the recurrence (6.13) is 
$$ OPT(i, j) = \max( OPT(i, j − 1), \max(1 + OPT(i, t − 1) + OPT(t + 1, j − 1)))$$
where for the $\max(1 + OPT(i, t − 1) + OPT(t + 1, j − 1))$ we use the function ``max_over_t`` we defined above. 
``n`` is the number of bases in the RNA primary structure (remember the ``len`` internal function)

In [11]:
def Tabulate_RNA(seq):
    # Below we will instantiate a matrix of nan values with the same dimensions as the length of our sequence.
    # We will then replace all nan values with 0 where i>=j-4
    
    M = np.empty((len(seq), len(seq)))
    M[:] = np.NaN
    
    for i in range(len(seq)):
        for j in range(len(seq)):
            if i >= j-4:
                M[i,j] = 0

    # Now that we have prepared our matrix for tabulation, we will loop through the indices of the array
    # starting with the smallest sub-sequences possible and progressing until we can calculate the optimal
    # value for the entire sequence
                
    for k in range(5, len(seq)):
        for i in range(0, len(seq) - k):
            j = i+k
            t, m = max_over_t(i, j, M, seq)
            M[i,j] = max(M[i,j-1], m)
            
    return M

## Look-up

Once we obtain the table (array) for the optimal value function we would like to look-up the list of all allowable pairs corresponding to the optimal value. 

The function List_Pairs takes as input
* ``M``, the array containing the tabulated optimal value function
* ``seq``, the string with the sequence of RNA bases
* ``i`` and ``j``, the indexes that tell which fragment of RNA to look at
* ``l``, the list of all allowable pairs corresponding to the optimal value.

and returns ``l`` 

Initially, ``i`` and ``j`` should be set to the whole RNA bases sequence and ``l`` to an empty list.
This is the blueprint of the function.

```
    List_Pairs(M,seq,i,j,l):
    Extract from M the maximum number of pairs maxpairs (done already in the code snippet)  
    if len(l)< maxpairs:
        if M[i,j] == M[i,j-1]
            List_Pairs(M,seq,i,j-1,l)
        else
            find the t that maximise 1 + M[i, t − 1] + M[t + 1, j − 1]
            Append t to the list l
            if M[t+1,j]>0
                List_Pairs(M,seq,t+1,j,l)
            if M[i,max(t-1,i)]>0
                List_Pairs(M,seq,i,t-1,l)
    return l
 ```
    
To find the t that maximise ``1 + M[i, t − 1] + M[t + 1, j − 1]`` use the ``max_over_t`` function.

In [12]:
def List_Pairs(M,seq,i,j,l):
    n = len(seq)
    maxpairs = int(M[0,n-1])
    
    if len(l) < maxpairs:
        if M[i,j] == M[i, j-1]:
            List_Pairs(M, seq, i, j-1, l)
        else:
            t, m = max_over_t(i, j, M, seq)
            l.append([t,j])
            if M[t+1, j]>0:
                List_Pairs(M, seq, t+1, j, l)
            if M[i, max(t-1, i)]>0:
                List_Pairs(M, seq, i, t-1, l)
    return l

## RNA base pairing

Finally, one can put all together to work out a few examples: 

In [13]:
seq = 'UGUACCGGUAGUACA'

M = Tabulate_RNA(seq)
M_ = M[:-5,5:]
print(M_)
l=[]
l=List_Pairs(M,seq,0,len(seq)-1,l)
print(l)

[[0. 0. 0. 1. 2. 2. 2. 3. 4. 5.]
 [0. 0. 0. 1. 2. 2. 2. 3. 4. 4.]
 [0. 0. 0. 1. 2. 2. 2. 3. 3. 3.]
 [0. 0. 0. 1. 1. 1. 2. 2. 2. 2.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0, 14], [1, 13], [2, 12], [3, 11], [4, 10]]


In [14]:
seq = 'CAGAUCGGCGAUACGAGCAUAGCAAUGCUAAGCGAGCUUAGCUGCA'

M = Tabulate_RNA(seq)
M_ = M[:-5,5:]
print(M_)
l=[]
l=List_Pairs(M,seq,0,len(seq)-1,l)
print(l)

[[ 0.  1.  1. ... 14. 14. 14.]
 [ 0.  0.  0. ... 13. 14. 14.]
 [ 0.  0.  0. ... 13. 14. 14.]
 ...
 [ 0.  0.  0. ...  0.  0.  1.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]]
[[15, 42], [16, 41], [27, 40], [28, 39], [29, 38], [30, 37], [31, 36], [17, 26], [18, 25], [19, 24], [0, 14], [2, 13], [3, 11], [4, 10]]
